In [3]:
import os
import tempfile
import numpy as np
import pandas as pd
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import pickle
import math

In [4]:
train_dir = 'data/train'
# 비디오 파일 목록과 태그를 포함하는 리스트를 만드는 함수
def create_data_list(data_dir):
    data_list = []
    # data_dir 안의 각 디렉토리에 대해 반복
    for item in os.listdir(data_dir):
        item_path = os.path.join(data_dir, item)  # 아이템의 전체 경로
        # 해당 경로가 디렉토리인지 확인
        if os.path.isdir(item_path):
            # 디렉토리 내의 모든 파일을 나열
            for file_name in os.listdir(item_path):
                # 파일이 .mp4 파일인지 확인
                if file_name.endswith('.jpg'):
                    # 리스트에 태그와 파일 경로를 추가
                    data_list.append((item, str(data_dir+'/'+item)+'/'+file_name))
    return data_list

# 함수를 사용해서 리스트를 생성
train_list = create_data_list(train_dir)
# 리스트에서 데이터프레임을 생성
train_df = pd.DataFrame(data=train_list, columns=['tag', 'image_name'])
# 필요한 경우 열 순서를 수정
train_df = train_df.loc[:, ['tag', 'image_name']]
# 데이터프레임을 CSV 파일로 저장
train_file_path = 'train.csv'
train_df.to_csv(train_file_path, encoding='utf-8-sig', index=False)
train_df = pd.read_csv("train.csv")
print(f"Total video for training: {len(train_df)}")

Total video for training: 1667


In [5]:
train_df

,tag,image_name
0,belly,data/train/belly/20231206_121209_001.jpg
1,belly,data/train/belly/20231206_121209_002.jpg
2,belly,data/train/belly/20231206_121209_003.jpg
3,belly,data/train/belly/20231206_121209_004.jpg
4,belly,data/train/belly/20231206_121209_005.jpg
...,...,...
1662,where,data/train/where/20231206_115431_012.jpg
1663,where,data/train/where/20231206_115431_013.jpg
1664,where,data/train/where/20231206_115431_014.jpg
1665,where,data/train/where/20231206_115431_015.jpg


In [6]:
# 손가락 각도 계산 함수
def calculate_angles(hand_landmarks, image_shape):
    joint = np.zeros((21, 3))
    for j, lm in enumerate(hand_landmarks.landmark):
        joint[j] = [lm.x * image_shape[1], lm.y * image_shape[0], lm.z]
    
    # 벡터 계산
    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
    v = v2 - v1
    # 정규화
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 각도 계산
    angle = np.arccos(np.einsum('nt,nt->n', v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
    angle = np.degrees(angle)
    
    return angle

In [7]:
def calculate_pose_angles(pose_landmarks, image_shape):
    joint = np.zeros((33, 3))
    for j, lm in enumerate(pose_landmarks.landmark):
        joint[j] = [lm.x * image_shape[1], lm.y * image_shape[0], lm.z]

    # 팔 각도 계산: 어깨(11, 12), 팔꿈치(13, 14), 손목(15, 16) 랜드마크 사용
    v1 = joint[[11, 13, 12, 14], :]  # 어깨와 팔꿈치
    v2 = joint[[13, 15, 14, 16], :]  # 팔꿈치와 손목
    v = v2 - v1
    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

    # 양팔의 각도 계산
    arm_angles = np.arccos(np.einsum('nt,nt->n', v[[0, 2], :], v[[1, 3], :]))
    arm_angles = np.degrees(arm_angles)

    return arm_angles

In [6]:
df_hands = pd.DataFrame()
# 이미지로부터 학습 데이터 생성 및 DataFrame에 추가하는 함수
def create_training_data_from_image(image_path, label):
    global df_hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
    
    # 이미지 파일 불러오기
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    pose_result = pose.process(img)

    # 양손의 랜드마크가 감지되었는지 확인하고 데이터 처리
    # 각 손에 대해 21개의 랜드마크에서 15개의 각도를 계산하므로 총 30개의 각도 데이터가 필요
    hand_data_row = [0] * 30  # 기본값으로 0으로 채워진 리스트를 생성
    arm_data_row = [0] * 2   # 팔 각도 데이터
    if result.multi_hand_landmarks:
        for i, hand_landmarks in enumerate(result.multi_hand_landmarks):
            angles = calculate_angles(hand_landmarks, img.shape[:2])
            hand_data_row[i*15:(i+1)*15] = angles
    if pose_result.pose_landmarks:
        arm_angles = calculate_pose_angles(pose_result.pose_landmarks, img.shape[:2])
        arm_data_row = arm_angles.tolist()

    # DataFrame에 행 추가    
    df_hands = pd.concat([df_hands, pd.DataFrame([hand_data_row + arm_data_row + [label]])], ignore_index=True)

    return df_hands

# 특정 폴더의 모든 이미지를 처리하고 CSV 파일로 저장하는 함수
def process_folder(folder_path, label):
    df_folder = pd.DataFrame()

    # 폴더 내 모든 이미지 처리
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        df_folder = create_training_data_from_image(image_path, label)

    # 컬럼 이름 설정 (30개의 각도 + 1개의 라벨)
    angle_columns = [f'angle_{i}' for i in range(30)] + ['arm_1', 'arm_2']
    df_folder.columns = angle_columns + ['label']

    # CSV 파일로 저장
    df_folder.to_csv(f'{label}_data.csv', index=False)

# 'data/train' 폴더 내 모든 하위 폴더를 순회
base_folder = 'data/train'
for label_folder in os.listdir(base_folder):
    folder_path = base_folder+"/"+label_folder
    if os.path.isdir(folder_path):
        print(folder_path)
        process_folder(folder_path, label_folder)
        

data/train/where


In [8]:
df_hands = pd.read_csv("final_training_data.csv")

In [9]:
df_hands

,angle_0,angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,...,angle_23,angle_24,angle_25,angle_26,angle_27,angle_28,angle_29,arm_1,arm_2,label
0,17.626396,5.989130,10.317576,15.149633,1.447276,0.320342,4.348895,1.877061,0.322290,14.482118,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.644514,105.876376,belly
1,15.812271,6.516015,11.230466,20.231538,1.977787,1.692256,4.297915,3.216543,0.189223,16.986444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.294668,108.264424,belly
2,15.429004,5.735022,10.687926,25.062960,0.247285,0.075435,6.428805,1.805776,1.141801,15.441685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.183517,109.213746,belly
3,17.752119,5.297128,9.918691,23.915642,0.235788,0.838819,6.195439,2.406301,1.803850,15.783006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.623855,110.851940,belly
4,15.287326,4.806262,8.508105,24.556346,0.314952,1.951877,5.933526,0.781696,2.717788,15.666043,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.922378,109.518135,belly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,38.048428,31.647939,18.349552,10.369115,37.800869,82.767777,7.940961,20.003313,30.197273,0.541812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.992469,18.504014,lost
1663,40.751749,28.342227,16.928919,7.832830,43.809296,72.226251,6.284269,19.466769,23.628273,0.273331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.786114,18.973384,lost
1664,37.351928,28.519518,25.886782,11.809793,39.852882,69.911474,8.977844,21.766992,23.631987,4.227289,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.934653,21.255231,lost
1665,57.287378,13.314084,29.970087,8.683483,12.813907,19.811492,3.037944,24.017960,15.148014,7.102899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.265476,47.377795,lost


In [10]:
X_train = df_hands.drop('label', axis =1)
y_train = df_hands['label']

In [11]:
X_train = pd.get_dummies(X_train)

In [12]:
X_train

,angle_0,angle_1,angle_2,angle_3,angle_4,angle_5,angle_6,angle_7,angle_8,angle_9,...,angle_22,angle_23,angle_24,angle_25,angle_26,angle_27,angle_28,angle_29,arm_1,arm_2
0,17.626396,5.989130,10.317576,15.149633,1.447276,0.320342,4.348895,1.877061,0.322290,14.482118,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.644514,105.876376
1,15.812271,6.516015,11.230466,20.231538,1.977787,1.692256,4.297915,3.216543,0.189223,16.986444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,51.294668,108.264424
2,15.429004,5.735022,10.687926,25.062960,0.247285,0.075435,6.428805,1.805776,1.141801,15.441685,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.183517,109.213746
3,17.752119,5.297128,9.918691,23.915642,0.235788,0.838819,6.195439,2.406301,1.803850,15.783006,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.623855,110.851940
4,15.287326,4.806262,8.508105,24.556346,0.314952,1.951877,5.933526,0.781696,2.717788,15.666043,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.922378,109.518135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,38.048428,31.647939,18.349552,10.369115,37.800869,82.767777,7.940961,20.003313,30.197273,0.541812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.992469,18.504014
1663,40.751749,28.342227,16.928919,7.832830,43.809296,72.226251,6.284269,19.466769,23.628273,0.273331,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.786114,18.973384
1664,37.351928,28.519518,25.886782,11.809793,39.852882,69.911474,8.977844,21.766992,23.631987,4.227289,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.934653,21.255231
1665,57.287378,13.314084,29.970087,8.683483,12.813907,19.811492,3.037944,24.017960,15.148014,7.102899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.265476,47.377795


In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [21]:
final_lgbm_model = LGBMClassifier(random_state=200, n_jobs=-1, reg_alpha=0.9, reg_lambda=0.2, n_estimators=200)
final_lgbm_model.fit(X_train, y_train)
result = cross_val_score(final_lgbm_model,
                          X_train,
                          y_train,
                          cv = 5)
result.mean()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7575
[LightGBM] [Info] Number of data points in the train set: 1667, number of used features: 32
[LightGBM] [Info] Start training from score -2.288882
[LightGBM] [Info] Start training from score -2.294817
[LightGBM] [Info] Start training from score -2.300787
[LightGBM] [Info] Start training from score -2.306793
[LightGBM] [Info] Start training from score -2.331185
[LightGBM] [Info] Start training from score -2.300787
[LightGBM] [Info] Start training from score -2.318914
[LightGBM] [Info] Start training from score -2.282982
[LightGBM] [Info] Start training from score -2.312835
[LightGBM] [Info] Start training from score -2.288882
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000894 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6945
[LightGBM] [Info] Number of data points in the train set: 1334, number of used features: 32
[LightGBM] [Info] Start training from score -2.283282
[LightGBM] [Info] Start training from score -2.298097
[LightGBM] [Info] Start training from score -2.298097
[LightGBM] [Info] Start training from score -2.305588
[LightGBM] [Info] Start training from score -2.336125
[LightGBM] [Info] Start training from score -2.305588
[LightGBM] [Info] Start training from score -2.313135
[LightGBM] [Info] Start training from score -2.283282
[LightGBM] [Info] Start training from score -2.313135
[LightGBM] [Info] Start training from score -2.290662
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

0.9801909694125264

In [22]:
final_lgbm_model.booster_.save_model('lgbm_model.txt')

In [23]:
"""cb_model = CatBoostClassifier()
cb_param = {"depth" : [4,6,8,10], #트리의 깊이
          "iterations" : [250,100,500,1000], #학습 반복횟수
          "learning_rate" : [0.001,0.01,0.1,0.2,0.3], #학습률
          "l2_leaf_reg" : [2,5,10,20,30], #L2규제 하이퍼파라미터
          "border_count" : [254]
          }
cb_grid = GridSearchCV (cb_model, cb_param, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
# scoring - 모델 성능을 평가하는데 사용할 지표(정확도)
# refit - 최적 하이퍼파라미터를 찾은 후 모델을 재학습
cb_grid.fit(X_train,y_train)
print('최고 평균 정확도 수치: {:.4f}'.format(cat_grid.best_score_))
print('최적 하이퍼 파라미터: ', cb_grid.best_params_)"""
# 얜 성능이 더 줄어들길래 다 기본값으로 
final_cb_model = CatBoostClassifier()
final_cb_model.fit(X_train, y_train)
result = cross_val_score(final_cb_model,
                          X_train,
                          y_train,
                          cv = 5)
result.mean()
# 0.875

Learning rate set to 0.081216
0:	learn: 1.8485532	total: 167ms	remaining: 2m 47s
1:	learn: 1.5563844	total: 192ms	remaining: 1m 35s
2:	learn: 1.3320995	total: 218ms	remaining: 1m 12s
3:	learn: 1.1615434	total: 244ms	remaining: 1m
4:	learn: 1.0370777	total: 269ms	remaining: 53.6s
5:	learn: 0.9340208	total: 296ms	remaining: 49s
6:	learn: 0.8426682	total: 321ms	remaining: 45.5s
7:	learn: 0.7616020	total: 345ms	remaining: 42.8s
8:	learn: 0.6933696	total: 370ms	remaining: 40.7s
9:	learn: 0.6340905	total: 394ms	remaining: 39s
10:	learn: 0.5815052	total: 427ms	remaining: 38.4s
11:	learn: 0.5347100	total: 476ms	remaining: 39.2s
12:	learn: 0.4926667	total: 506ms	remaining: 38.5s
13:	learn: 0.4542845	total: 534ms	remaining: 37.6s
14:	learn: 0.4213756	total: 560ms	remaining: 36.8s
15:	learn: 0.3920340	total: 585ms	remaining: 36s
16:	learn: 0.3630854	total: 610ms	remaining: 35.3s
17:	learn: 0.3378745	total: 636ms	remaining: 34.7s
18:	learn: 0.3148044	total: 661ms	remaining: 34.1s
19:	learn: 0.2949

165:	learn: 0.0088880	total: 4.86s	remaining: 24.4s
166:	learn: 0.0088618	total: 4.89s	remaining: 24.4s
167:	learn: 0.0087955	total: 4.91s	remaining: 24.3s
168:	learn: 0.0086737	total: 4.94s	remaining: 24.3s
169:	learn: 0.0085821	total: 4.97s	remaining: 24.3s
170:	learn: 0.0084773	total: 4.99s	remaining: 24.2s
171:	learn: 0.0084531	total: 5.02s	remaining: 24.2s
172:	learn: 0.0083833	total: 5.05s	remaining: 24.1s
173:	learn: 0.0082668	total: 5.07s	remaining: 24.1s
174:	learn: 0.0082438	total: 5.1s	remaining: 24s
175:	learn: 0.0081734	total: 5.13s	remaining: 24s
176:	learn: 0.0080935	total: 5.15s	remaining: 24s
177:	learn: 0.0080630	total: 5.18s	remaining: 23.9s
178:	learn: 0.0080055	total: 5.21s	remaining: 23.9s
179:	learn: 0.0079044	total: 5.24s	remaining: 23.9s
180:	learn: 0.0078679	total: 5.26s	remaining: 23.8s
181:	learn: 0.0077993	total: 5.29s	remaining: 23.8s
182:	learn: 0.0077170	total: 5.32s	remaining: 23.7s
183:	learn: 0.0076843	total: 5.34s	remaining: 23.7s
184:	learn: 0.00763

330:	learn: 0.0036512	total: 9.7s	remaining: 19.6s
331:	learn: 0.0036314	total: 9.72s	remaining: 19.6s
332:	learn: 0.0036186	total: 9.75s	remaining: 19.5s
333:	learn: 0.0036025	total: 9.78s	remaining: 19.5s
334:	learn: 0.0035842	total: 9.8s	remaining: 19.5s
335:	learn: 0.0035669	total: 9.83s	remaining: 19.4s
336:	learn: 0.0035484	total: 9.86s	remaining: 19.4s
337:	learn: 0.0035323	total: 9.88s	remaining: 19.4s
338:	learn: 0.0035137	total: 9.91s	remaining: 19.3s
339:	learn: 0.0035058	total: 9.94s	remaining: 19.3s
340:	learn: 0.0034931	total: 9.96s	remaining: 19.3s
341:	learn: 0.0034884	total: 9.99s	remaining: 19.2s
342:	learn: 0.0034818	total: 10s	remaining: 19.2s
343:	learn: 0.0034558	total: 10s	remaining: 19.1s
344:	learn: 0.0034473	total: 10.1s	remaining: 19.1s
345:	learn: 0.0034345	total: 10.1s	remaining: 19.1s
346:	learn: 0.0034249	total: 10.1s	remaining: 19s
347:	learn: 0.0034109	total: 10.1s	remaining: 19s
348:	learn: 0.0033930	total: 10.2s	remaining: 19s
349:	learn: 0.0033883	to

492:	learn: 0.0021656	total: 14.5s	remaining: 14.9s
493:	learn: 0.0021634	total: 14.5s	remaining: 14.9s
494:	learn: 0.0021613	total: 14.5s	remaining: 14.8s
495:	learn: 0.0021586	total: 14.6s	remaining: 14.8s
496:	learn: 0.0021514	total: 14.6s	remaining: 14.8s
497:	learn: 0.0021471	total: 14.6s	remaining: 14.7s
498:	learn: 0.0021366	total: 14.7s	remaining: 14.7s
499:	learn: 0.0021321	total: 14.7s	remaining: 14.7s
500:	learn: 0.0021306	total: 14.7s	remaining: 14.6s
501:	learn: 0.0021289	total: 14.7s	remaining: 14.6s
502:	learn: 0.0021238	total: 14.8s	remaining: 14.6s
503:	learn: 0.0021198	total: 14.8s	remaining: 14.5s
504:	learn: 0.0021100	total: 14.8s	remaining: 14.5s
505:	learn: 0.0021050	total: 14.8s	remaining: 14.5s
506:	learn: 0.0020982	total: 14.9s	remaining: 14.4s
507:	learn: 0.0020936	total: 14.9s	remaining: 14.4s
508:	learn: 0.0020882	total: 14.9s	remaining: 14.4s
509:	learn: 0.0020818	total: 14.9s	remaining: 14.4s
510:	learn: 0.0020768	total: 15s	remaining: 14.3s
511:	learn: 0.

654:	learn: 0.0015636	total: 19.1s	remaining: 10.1s
655:	learn: 0.0015628	total: 19.1s	remaining: 10s
656:	learn: 0.0015581	total: 19.2s	remaining: 10s
657:	learn: 0.0015546	total: 19.2s	remaining: 9.98s
658:	learn: 0.0015518	total: 19.2s	remaining: 9.95s
659:	learn: 0.0015492	total: 19.3s	remaining: 9.92s
660:	learn: 0.0015436	total: 19.3s	remaining: 9.89s
661:	learn: 0.0015402	total: 19.3s	remaining: 9.87s
662:	learn: 0.0015376	total: 19.4s	remaining: 9.84s
663:	learn: 0.0015355	total: 19.4s	remaining: 9.82s
664:	learn: 0.0015334	total: 19.5s	remaining: 9.8s
665:	learn: 0.0015322	total: 19.5s	remaining: 9.78s
666:	learn: 0.0015294	total: 19.5s	remaining: 9.75s
667:	learn: 0.0015271	total: 19.6s	remaining: 9.72s
668:	learn: 0.0015223	total: 19.6s	remaining: 9.69s
669:	learn: 0.0015172	total: 19.6s	remaining: 9.66s
670:	learn: 0.0015135	total: 19.6s	remaining: 9.63s
671:	learn: 0.0015103	total: 19.7s	remaining: 9.59s
672:	learn: 0.0015066	total: 19.7s	remaining: 9.56s
673:	learn: 0.001

...

364:	learn: 0.0036184	total: 10.7s	remaining: 18.6s
365:	learn: 0.0036040	total: 10.7s	remaining: 18.6s
366:	learn: 0.0035909	total: 10.7s	remaining: 18.5s
367:	learn: 0.0035769	total: 10.8s	remaining: 18.5s
368:	learn: 0.0035656	total: 10.8s	remaining: 18.4s
369:	learn: 0.0035564	total: 10.8s	remaining: 18.4s
370:	learn: 0.0035511	total: 10.8s	remaining: 18.4s
371:	learn: 0.0035332	total: 10.9s	remaining: 18.3s
372:	learn: 0.0035216	total: 10.9s	remaining: 18.3s
373:	learn: 0.0035036	total: 10.9s	remaining: 18.3s
374:	learn: 0.0034914	total: 10.9s	remaining: 18.2s
375:	learn: 0.0034868	total: 11s	remaining: 18.2s
376:	learn: 0.0034770	total: 11s	remaining: 18.2s
377:	learn: 0.0034630	total: 11s	remaining: 18.1s
378:	learn: 0.0034530	total: 11.1s	remaining: 18.1s
379:	learn: 0.0034483	total: 11.1s	remaining: 18.1s
380:	learn: 0.0034367	total: 11.1s	remaining: 18s
381:	learn: 0.0034227	total: 11.1s	remaining: 18s
382:	learn: 0.0034093	total: 11.2s	remaining: 18s
383:	learn: 0.0034027	to

527:	learn: 0.0023535	total: 15.4s	remaining: 13.8s
528:	learn: 0.0023516	total: 15.5s	remaining: 13.8s
529:	learn: 0.0023449	total: 15.5s	remaining: 13.7s
530:	learn: 0.0023423	total: 15.5s	remaining: 13.7s
531:	learn: 0.0023375	total: 15.5s	remaining: 13.7s
532:	learn: 0.0023332	total: 15.6s	remaining: 13.6s
533:	learn: 0.0023307	total: 15.6s	remaining: 13.6s
534:	learn: 0.0023242	total: 15.6s	remaining: 13.6s
535:	learn: 0.0023206	total: 15.7s	remaining: 13.6s
536:	learn: 0.0023179	total: 15.7s	remaining: 13.5s
537:	learn: 0.0023146	total: 15.7s	remaining: 13.5s
538:	learn: 0.0023114	total: 15.7s	remaining: 13.5s
539:	learn: 0.0023075	total: 15.8s	remaining: 13.4s
540:	learn: 0.0023027	total: 15.8s	remaining: 13.4s
541:	learn: 0.0023007	total: 15.8s	remaining: 13.4s
542:	learn: 0.0022992	total: 15.8s	remaining: 13.3s
543:	learn: 0.0022947	total: 15.9s	remaining: 13.3s
544:	learn: 0.0022890	total: 15.9s	remaining: 13.3s
545:	learn: 0.0022845	total: 15.9s	remaining: 13.2s
546:	learn: 

691:	learn: 0.0017140	total: 20.2s	remaining: 8.99s
692:	learn: 0.0017099	total: 20.2s	remaining: 8.97s
693:	learn: 0.0017062	total: 20.3s	remaining: 8.95s
694:	learn: 0.0017051	total: 20.3s	remaining: 8.92s
695:	learn: 0.0017028	total: 20.4s	remaining: 8.89s
696:	learn: 0.0017004	total: 20.4s	remaining: 8.86s
697:	learn: 0.0016970	total: 20.4s	remaining: 8.83s
698:	learn: 0.0016935	total: 20.4s	remaining: 8.8s
699:	learn: 0.0016914	total: 20.5s	remaining: 8.77s
700:	learn: 0.0016889	total: 20.5s	remaining: 8.74s
701:	learn: 0.0016871	total: 20.5s	remaining: 8.71s
702:	learn: 0.0016835	total: 20.5s	remaining: 8.68s
703:	learn: 0.0016813	total: 20.6s	remaining: 8.64s
704:	learn: 0.0016783	total: 20.6s	remaining: 8.62s
705:	learn: 0.0016753	total: 20.6s	remaining: 8.58s
706:	learn: 0.0016733	total: 20.6s	remaining: 8.55s
707:	learn: 0.0016721	total: 20.7s	remaining: 8.52s
708:	learn: 0.0016678	total: 20.7s	remaining: 8.49s
709:	learn: 0.0016656	total: 20.7s	remaining: 8.46s
710:	learn: 0

858:	learn: 0.0013491	total: 25s	remaining: 4.11s
859:	learn: 0.0013479	total: 25.1s	remaining: 4.08s
860:	learn: 0.0013466	total: 25.1s	remaining: 4.05s
861:	learn: 0.0013458	total: 25.2s	remaining: 4.03s
862:	learn: 0.0013449	total: 25.2s	remaining: 4s
863:	learn: 0.0013431	total: 25.2s	remaining: 3.97s
864:	learn: 0.0013421	total: 25.3s	remaining: 3.94s
865:	learn: 0.0013408	total: 25.3s	remaining: 3.92s
866:	learn: 0.0013388	total: 25.3s	remaining: 3.89s
867:	learn: 0.0013375	total: 25.4s	remaining: 3.86s
868:	learn: 0.0013368	total: 25.4s	remaining: 3.83s
869:	learn: 0.0013348	total: 25.4s	remaining: 3.8s
870:	learn: 0.0013339	total: 25.4s	remaining: 3.77s
871:	learn: 0.0013313	total: 25.5s	remaining: 3.74s
872:	learn: 0.0013290	total: 25.5s	remaining: 3.71s
873:	learn: 0.0013282	total: 25.5s	remaining: 3.68s
874:	learn: 0.0013270	total: 25.5s	remaining: 3.65s
875:	learn: 0.0013250	total: 25.6s	remaining: 3.62s
876:	learn: 0.0013237	total: 25.6s	remaining: 3.59s
877:	learn: 0.0013

0.993997590404776

In [27]:
final_cb_model.save_model('cb_model.cbm')

In [28]:
from lightgbm import Booster
lgbm_model = Booster(model_file='lgbm_model.txt')
cb_model = CatBoostClassifier()  # 또는 CatBoostRegressor
cb_model.load_model('cb_model.cbm')

In [43]:
import cv2
import mediapipe as mp
import numpy as np
import catboost as cb

# 필요한 Mediapipe 솔루션을 초기화합니다.
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 이미지 파일 읽기 (이 부분에서 파일 경로를 수정하세요)
img = cv2.imread('data/train/where/20231206_115419_073.jpg')

# 이미지 처
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
result = hands.process(img)
pose_result = pose.process(img)
img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

if pose_result.pose_landmarks:
    arm_angles = calculate_pose_angles(pose_result.pose_landmarks, img.shape)
    
if result.multi_hand_landmarks:
    for res in result.multi_hand_landmarks:
        joint = np.zeros((21, 3))
        for j, lm in enumerate(res.landmark):
            joint[j] = [lm.x, lm.y, lm.z]

        # 벡터 계산
        v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
        v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
        v = v2 - v1
        v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

        # 각도 계산
        angle = np.arccos(np.einsum('nt,nt->n',
            v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
            v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
        angle = np.degrees(angle)

        # NaN 값이 있을 경우 0으로 대체
        angle = np.nan_to_num(angle)

        # 데이터 차원 맞추기
        data = np.concatenate((angle, [0]*(30-len(angle)))) if len(angle) < 30 else angle
        data = np.concatenate((data, arm_angles)) if len(arm_angles) > 0 else data
        data = data.reshape(1, -1)  # 예측을 위해 데이터를 적절한 형태로 변환
        predicted_label = cb_model.predict(data)
        print(predicted_label)
        # 예측된 라벨을 화면에 표시
        org = (int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0]))
        cv2.putText(img, text=str(predicted_label[0]), org=(org[0], org[1] + 20), 
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

        # 손 랜드마크 그리기
        mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

# 화면에 이미지 표시
cv2.imshow('img', img)
cv2.waitKey(0)  # 키를 누르면 창이 닫힙니다.

# 사용 종료 후 자원 해제
cv2.destroyAllWindows()

[['sick']]


In [46]:
# 필요한 Mediapipe 솔루션을 초기화합니다.
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, img = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue

    # 이미지 처리
    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    pose_result = pose.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    hands_data = np.zeros((2, 30))
    
    if result.multi_hand_landmarks:
        for i, res in enumerate(result.multi_hand_landmarks):
            if i>= 2:
                break
            
            joint = np.zeros((21, 3))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z]

            # 벡터 계산
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
            v = v2 - v1
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # 각도 계산
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))
            angle = np.degrees(angle)

            # NaN 값이 있을 경우 0으로 대체
            angle = np.nan_to_num(angle)
            
            hands_data[i, : len(angle)]=angle
            
    # 데이터 차원 맞추기
    data = np.concatenate((hands_data[0], hands_data[1]))

    if pose_result.pose_landmarks:
        arm_angles = calculate_pose_angles(pose_result.pose_landmarks, img.shape)
        data = np.concatenate((data, arm_angles)) if len(arm_angles) > 0 else data
        
    data = data.reshape(1,-1)
    predicted_label = cb_model.predict(data)
    print(predicted_label)
    
    # 예측된 라벨을 화면에 표시 (예시)
    cv2.putText(img, text=str(predicted_label[0]), org=(50, 50),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
    
    # 손 랜드마크 그리기
    if result.multi_hand_landmarks:
        for res in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
    
    # 화면에 이미지 표시
    cv2.imshow('img', img)
    if cv2.waitKey(30) == 49:  # 1번 키를 누르면 종료
        break

# 사용 종료 후 자원 해제
cap.release()
cv2.destroyAllWindows()

[['lost']]
[['lost']]
[['sick']]
[['lost']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['sick']]
[['sick']]
[['sick']]
[['where']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['lost']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['lost']]
[['sick']]
[['lost']]
[['sick']]
[['lost']]
[['down']]
[['down']]
[['down']]
[['down']]
[['down']]
[['down']]
[['lost']]
[['down']]
[['down']]
[['down']

[['where']]
[['where']]
[['belly']]
[['where']]
[['belly']]
[['where']]
[['belly']]
[['belly']]
[['belly']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[['sick']]
[